Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [1]:
pip install PySpark

Note: you may need to restart the kernel to use updated packages.


## Build Spark Session

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.getOrCreate()


## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [4]:
df = spark.read.csv('train.csv', inferSchema=True, header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [5]:
df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

**Show 5 rows.**

In [6]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [7]:
df.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [8]:
df.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [9]:
df.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [10]:
df.createOrReplaceTempView("titanic")

In [11]:
from pyspark.sql.functions import *

In [12]:
#survived=df.select(col('Survived')).agg(sum('Survived'))

In [13]:
survived=spark.sql("""select sum(Survived)
from  titanic""")

**Display your result:**

In [14]:
survived.show()


+-------------+
|sum(Survived)|
+-------------+
|          342|
+-------------+



**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [15]:
sum_sur=survived.first()[0]
sum_sur
total_rec=df.count()


In [16]:
ratio=(sum_sur/total_rec)*100

In [17]:
print('surrvivred ratio is {:.2f} %'.format(ratio))

surrvivred ratio is 38.38 %


**Can you get the number of males and females?**


In [18]:

gender=df.groupBy('Sex').count().orderBy('Sex')

In [19]:
gender.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [20]:
spark.sql("""select avg(Survived) as male_survivors
from  titanic where Sex == 'male'  """).show()


spark.sql("""select avg(Survived) as female_survivors
from  titanic where Sex == 'female'  """).show()


spark.sql("""select sum(Survived) as male_survivors_num
from  titanic where Sex == 'male'  """).show()
spark.sql("""select sum(Survived) as female_survivors_num
from  titanic where Sex == 'female'  """).show()

+-------------------+
|     male_survivors|
+-------------------+
|0.18890814558058924|
+-------------------+

+------------------+
|  female_survivors|
+------------------+
|0.7420382165605095|
+------------------+

+------------------+
|male_survivors_num|
+------------------+
|               109|
+------------------+

+--------------------+
|female_survivors_num|
+--------------------+
|                 233|
+--------------------+



**Create temporary view PySpark:**

In [21]:
df.createOrReplaceTempView("titanic")

**How many people survived, and how many didn't survive? By SQL:**

In [22]:
spark.sql("""select count(Survived) as Survived
from  titanic where  Survived =1""").show()
spark.sql("""select count(Survived) as didnt_Survived
from  titanic where  Survived =0""").show()

+--------+
|Survived|
+--------+
|     342|
+--------+

+--------------+
|didnt_Survived|
+--------------+
|           549|
+--------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [23]:
spark.sql("""SELECT sum(case when `Sex` = 'male' then 1 else 0 end)/count(*) as male_ratio,
       sum(case when `Sex` = 'female' then 1 else 0 end)/count(*) as female_ratio
FROM titanic""").show()




+------------------+-------------------+
|        male_ratio|       female_ratio|
+------------------+-------------------+
|0.6475869809203143|0.35241301907968575|
+------------------+-------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [24]:
spark.sql("""select sum(Survived)/count(DISTINCT Pclass) as pcalss_ratio
from  titanic """).show()


+------------+
|pcalss_ratio|
+------------+
|       114.0|
+------------+



In [25]:
df.groupBy('Pclass').count().orderBy('Pclass').show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     2|  184|
|     3|  491|
+------+-----+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [26]:
test_df = spark.read.csv('test.csv', inferSchema=True, header=True)
merged_df = df.unionByName(test_df, allowMissingColumns=True)

**Display count:**

In [27]:
merged_df.count()

1329

**Can you define the number of null values in each column?**


In [28]:
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [29]:
null_values=merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).collect()
list1=[]
for i in zip(merged_df.columns,null_values[0]):
    if i[1]!=0:
        list1.append(i)


In [30]:
spark.createDataFrame(data=list1,schema=['col','null_count']).show()

+--------+----------+
|     col|null_count|
+--------+----------+
|     Age|       265|
|   Cabin|      1021|
|Embarked|         3|
+--------+----------+



In [31]:
#null_values.replace(0,null)
#null_values.show()

## Preprocessing 

**Create Temporary view PySpark:**

In [32]:
merged_df.createOrReplaceTempView("PySpark")

**Can you show the "name" column from your temporary table?**

In [33]:
spark.sql("""select Name
from  PySpark """).show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [34]:
import pyspark.sql.functions as F
merged_df = merged_df.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
merged_df.createOrReplaceTempView('combined')

**Display "Title" column and count "Title" column:**

In [35]:
title=spark.sql("""select Title,COUNT(Title) AS Title_count
from  combined GROUP BY Title """)
title.show()



+--------+-----------+
|   Title|Title_count|
+--------+-----------+
|     Don|          1|
|    Miss|        257|
|Countess|          2|
|     Col|          4|
|     Rev|          9|
|    Lady|          2|
|  Master|         56|
|     Mme|          1|
|    Capt|          2|
|      Mr|        786|
|      Dr|         11|
|     Mrs|        186|
|     Sir|          2|
|Jonkheer|          2|
|    Mlle|          4|
|   Major|          3|
|      Ms|          1|
+--------+-----------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [36]:
temp1=spark.sql("select Title ,count(Title) as sum from combined group by Title")
data_temp=list(temp1.select(temp1.Title).where(temp1.sum<10).toPandas()['Title'])
data_temp1=list(temp1.select(temp1.Title).where(temp1.sum>10).toPandas()['Title'])
titles_map1={i: 'title' for i in data_temp1}
titles_map={i: 'rare' for i in data_temp}
data_temp1=list(temp1.select(temp1.Title).where(temp1.sum>10).toPandas()['Title'])
titles_map1={i: 'title' for i in data_temp1}
titles_map={i: 'rare' for i in data_temp}
titles_map.update(titles_map1)
titles_map

{'Don': 'rare',
 'Countess': 'rare',
 'Col': 'rare',
 'Rev': 'rare',
 'Lady': 'rare',
 'Mme': 'rare',
 'Capt': 'rare',
 'Sir': 'rare',
 'Jonkheer': 'rare',
 'Mlle': 'rare',
 'Major': 'rare',
 'Ms': 'rare',
 'Miss': 'title',
 'Master': 'title',
 'Mr': 'title',
 'Dr': 'title',
 'Mrs': 'title'}

**Run the function:**

In [37]:
@udf(returnType=StringType()) 
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [38]:
 
merged_df.withColumn("impute_title", impute_title(F.col("Title"))) \
  .show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|impute_title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|       title|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|       title|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|       title|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|       title|
|          5|       0|     3|Allen, Mr. Willia...|  mal

**Display "Title" from table and group by "Title" column:**

In [39]:
spark.sql("select Title from combined group by Title").show()

+--------+
|   Title|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [40]:
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [41]:
mean_val = merged_df.select(mean(df['age'])).collect()
mean_value = mean_val[0][0]


**Fill missing with "age" mean:**

In [42]:
merged_df=merged_df.na.fill(mean_value,subset=['Age'])
merged_df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Miss|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
only showing top 3 rows



In [43]:
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0| 1021|       3|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [44]:
grouped_Embarked=spark.sql("""select Embarked ,count(Embarked) as count from combined GROUP BY Embarked ORDER BY count DESC """)




**Show "groupped_Embarked" your variable:**

In [45]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    0|
+--------+-----+



**Get max of groupped_Embarked:** 

In [46]:

grouped_Embarked.agg({"count": "max"}).collect()[0]

Row(max(count)=962)

**Fill missing values with max 'S' of grouped_Embarked:**

In [47]:
merged_df=merged_df.na.fill("S",subset=['Embarked'])
merged_df.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

In [48]:
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0| 1021|       0|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [49]:

merged_df=merged_df.withColumn("Cabin",col('Cabin').substr(1,1))


**Show the result:**

In [50]:
merged_df.select('Cabin').distinct().show()

+-----+
|Cabin|
+-----+
|    F|
| null|
|    E|
|    T|
|    B|
|    D|
|    C|
|    A|
|    G|
+-----+



**Create the temporary view:**

In [51]:
merged_df.createOrReplaceTempView("temp")

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [52]:
spark.sql("""select Cabin ,count(Cabin) as count from temp GROUP BY Cabin ORDER BY count DESC """).show()



+-----+-----+
|Cabin|count|
+-----+-----+
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
| null|    0|
+-----+-----+



**Fill missing values with "U":**

In [53]:
merged_df=merged_df.na.fill("U",subset=['Cabin'])

In [54]:
merged_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in merged_df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Title|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|    0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [55]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,OneHotEncoder

In [56]:
categoricalCols = [field for (field, dataType) in merged_df.dtypes
                   if dataType == "string" and field not in ['Ticket','Name','Embarked','Title']]

indexOutputCols = [x + "_Index" for x in categoricalCols]
oheOutputCols = [x + "_OHE" for x in categoricalCols]
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols)
categoricalCols

['Sex', 'Cabin']

In [57]:

numericCols = [field for (field,dataType) in merged_df.dtypes
              if ((dataType!='string')& (field not in ['Survived','PassengerId']))]
numericCols

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [58]:
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [59]:
from pyspark.ml.feature import VectorAssembler

In [60]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Sex_OHE', 'Cabin_OHE', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [61]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [62]:
x_train, X_test = merged_df.randomSplit([.8,.2],seed=42)

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [63]:
from pyspark.ml.classification import RandomForestClassifier

In [64]:
RM = RandomForestClassifier(labelCol='Survived',featuresCol='features')


In [65]:
pipeline=Pipeline(stages=[stringIndexer,oheEncoder,vecAssembler,RM])
pipe_mdl=pipeline.fit(x_train)
predict=pipe_mdl.transform(X_test)

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [66]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [67]:
evaluator=MulticlassClassificationEvaluator()
evaluator.setPredictionCol('prediction')
evaluator.setLabelCol('Survived')
evaluator.setMetricName('accuracy')
evaluator.evaluate(predict)

0.8170212765957446

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
